In [1]:
import dask.distributed
import dask.dataframe as dd
import pandas as pd
import numpy as np

In [2]:
client = dask.distributed.Client('localhost:8786')

In [16]:
client.restart()

<Client: scheduler='tcp://localhost:8786' processes=4 cores=4>

In [17]:
df = dd.read_parquet('/bigdata/citibike.parquet')
# df = client.persist(df.repartition(npartitions=20))

In [18]:
df.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970322,504,1 Ave & E 15 St,40.732220,-73.981659,16950,Customer,NaN,0
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002953,459,W 20 St & 11 Ave,40.746746,-74.007759,19816,Customer,NaN,0
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730286,-73.990768,237,E 11 St & 2 Ave,40.730473,-73.986725,14548,Subscriber,1980.0,2
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718941,-73.992661,499,Broadway & W 60 St,40.769154,-73.981918,16063,Customer,NaN,0
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734928,-73.992004,410,Suffolk St & Stanton St,40.720665,-73.985176,19213,Subscriber,1986.0,1


In [7]:
all_ids_in_dataset = df.start_station_id.unique().compute().values.tolist() + df.end_station_id.unique().compute().values.tolist()
all_ids_in_dataset.sort()

In [8]:
import urllib, json

In [9]:
zz = pd.DataFrame(json.loads(
        (urllib.request.urlopen('https://feeds.citibikenyc.com/stations/stations.json').read()).decode('utf-8')
    )['stationBeanList'])
zz.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,1,37,,72,,2017-04-19 12:56:59 PM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,27,6,,79,,2017-04-19 12:56:18 PM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,0,0,,82,,2017-04-13 12:05:52 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,3,Not In Service,False,0
3,,33,26,,83,,2017-04-19 12:58:16 PM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,12,26,,116,,2017-04-19 12:58:08 PM,40.741776,,-74.001497,,W 17 St & 8 Ave,,W 17 St & 8 Ave,1,In Service,False,39


In [20]:
df.start_station_id.compute().shape

(36902025,)

In [21]:
df_s = df[['start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude']]
df_e = df[['end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude']]

In [22]:
df_s = df_s[(df_s.start_station_latitude > 40.) & (df_s.start_station_latitude < 41.)]
df_s = df_s[(df_s.start_station_longitude + 74.0 > -0.25) & (df_s.start_station_longitude + 74.0 < 0.25)]

df_e = df_e[(df_e.end_station_latitude > 40.) & (df_e.end_station_latitude < 41.)]
df_e = df_e[(df_e.end_station_longitude + 74.0 > -0.25) & (df_e.end_station_longitude + 74.0 < 0.25)]


In [23]:
df_s.head()

,start_station_id,start_station_name,start_station_latitude,start_station_longitude
0,164,E 47 St & 2 Ave,40.753231,-73.970322
1,388,W 26 St & 10 Ave,40.749718,-74.002953
2,293,Lafayette St & E 8 St,40.730286,-73.990768
3,531,Forsyth St & Broome St,40.718941,-73.992661
4,382,University Pl & E 14 St,40.734928,-73.992004


In [26]:
a1 = df_s.groupby(['start_station_id', 'start_station_name']).mean().compute()
a2 = df_e.groupby(['end_station_id', 'end_station_name']).mean().compute()

In [27]:
b1 = a1.copy()
b2 = a2.copy()

In [28]:
b1.index = b1.index.rename(['id', 'name'])
b1.columns = ['lat', 'lon']
b2.index = b2.index.rename(['id', 'name'])
b2.columns = ['lat', 'lon']

In [83]:
c = b1.append(b2).drop_duplicates().reset_index().sort_values('id').reset_index(drop=True)

In [106]:
mm = (df[['start_station_id', 'start_station_name', 'end_station_id', 'end_station_name']])[(df.start_station_id==3239) | (df.end_station_id == 3239)].compute()
mm

,start_station_id,start_station_name,end_station_id,end_station_name
266052,3239,Bressler,3239,Bressler
268611,3239,Bressler,3239,Bressler
268802,3239,Bressler,3239,Bressler
269195,3239,Bressler,3239,Bressler
341927,3239,Bressler,3239,Bressler
343318,3239,Bressler,3239,Bressler
347861,3239,Bressler,3239,Bressler
365081,3239,Bressler,3239,Bressler
373718,3239,Bressler,3239,Bressler
374793,3239,Bressler,3239,Bressler


In [85]:
c.sort_values(['lat', 'lon'])

,id,name,lat,lon
1074,3239,Bressler,40.646539,-74.016588
1083,3245,Kiosk in a box Motivate,40.646677,-74.016298
734,3040,SSP Tech Workshop,40.646678,-74.016262
733,3040,SSP Tech Workshop,40.646679,-74.016269
116,255,NYCBS Depot - SSP,40.646765,-74.016513
117,255,NYCBS Depot - SSP,40.646766,-74.016510
731,3036,8D OPS 01,40.651489,-74.026636
1384,3423,West Drive & Prospect Park West,40.661064,-73.979458
1385,3423,West Drive & Prospect Park West,40.661066,-73.979451
1400,3432,Bike in Movie Night | Prospect Park Bandshell,40.662987,-73.976919


In [99]:
zz[['id', 'stationName', 'latitude', 'longitude']].sort_values(['latitude', 'longitude'])

,id,stationName,latitude,longitude
639,3423,West Drive & Prospect Park West,40.661063,-73.979453
530,3310,14 St & 7 Ave,40.663779,-73.983968
520,3300,Prospect Park West & 8 St,40.665147,-73.976376
526,3306,10 St & 7 Ave,40.666208,-73.981999
539,3319,14 St & 5 Ave,40.666287,-73.988951
533,3313,6 Ave & 12 St,40.666318,-73.985462
524,3304,6 Ave & 9 St,40.668127,-73.983776
573,3354,3 St & Prospect Park West,40.668132,-73.973638
544,3324,3 Ave & 14 St,40.668546,-73.993333
542,3322,12 St & 4 Ave,40.668603,-73.990439


In [89]:
import sklearn.neighbors

In [90]:
help(sklearn.neighbors.BallTree)

Help on class BallTree in module sklearn.neighbors.ball_tree:

class BallTree(BinaryTree)
 |  BallTree for fast generalized N-point problems
 |  
 |  BallTree(X, leaf_size=40, metric='minkowski', \**kwargs)
 |  
 |  Parameters
 |  ----------
 |  X : array-like, shape = [n_samples, n_features]
 |      n_samples is the number of points in the data set, and
 |      n_features is the dimension of the parameter space.
 |      Note: if X is a C-contiguous array of doubles then data will
 |      not be copied. Otherwise, an internal copy will be made.
 |  
 |  leaf_size : positive integer (default = 40)
 |      Number of points at which to switch to brute-force. Changing
 |      leaf_size will not affect the results of a query, but can
 |      significantly impact the speed of a query and the memory required
 |      to store the constructed tree.  The amount of memory needed to
 |      store the tree scales as approximately n_samples / leaf_size.
 |      For a specified ``leaf_size``, a leaf 

In [94]:
bt = sklearn.neighbors.BallTree(zz[['id', 'stationName', 'latitude', 'longitude']].ix[:, 2:].values, metric='euclidean')

In [114]:
list(zip(c.id, bt.query_radius(c.ix[:,['lat', 'lon']], 0.0005)))

[(72, array([0])),
 (72, array([0])),
 (79, array([1])),
 (79, array([1])),
 (82, array([2])),
 (82, array([2])),
 (83, array([3])),
 (83, array([3])),
 (116, array([4])),
 (116, array([4])),
 (119, array([5])),
 (119, array([5])),
 (120, array([6])),
 (120, array([6])),
 (127, array([7])),
 (127, array([7])),
 (128, array([8])),
 (128, array([8])),
 (137, array([9])),
 (137, array([9])),
 (143, array([10])),
 (143, array([10])),
 (144, array([11])),
 (144, array([11])),
 (146, array([12])),
 (146, array([12])),
 (147, array([13])),
 (147, array([13])),
 (150, array([14])),
 (150, array([14])),
 (151, array([15])),
 (151, array([15])),
 (152, array([16])),
 (152, array([16])),
 (153, array([17])),
 (153, array([17])),
 (157, array([18])),
 (157, array([18])),
 (160, array([], dtype=int64)),
 (160, array([], dtype=int64)),
 (161, array([19])),
 (161, array([19])),
 (164, array([20])),
 (164, array([20])),
 (167, array([21])),
 (167, array([21])),
 (168, array([22])),
 (168, array([22]))

In [118]:
pd.Series(bt.query_radius(c.ix[:,['lat', 'lon']], 0.0005))

0         [0]
1         [0]
2         [1]
3         [1]
4         [2]
5         [2]
6         [3]
7         [3]
8         [4]
9         [4]
10        [5]
11        [5]
12        [6]
13        [6]
14        [7]
15        [7]
16        [8]
17        [8]
18        [9]
19        [9]
20       [10]
21       [10]
22       [11]
23       [11]
24       [12]
25       [12]
26       [13]
27       [13]
28       [14]
29       [14]
        ...  
1383    [638]
1384    [639]
1385    [639]
1386    [640]
1387    [640]
1388    [641]
1389    [641]
1390    [642]
1391    [642]
1392    [643]
1393    [643]
1394    [644]
1395    [644]
1396    [645]
1397    [645]
1398    [646]
1399    [646]
1400       []
1401       []
1402    [647]
1403    [647]
1404    [648]
1405    [648]
1406    [649]
1407    [649]
1408    [650]
1409    [650]
1410    [169]
1411    [651]
1412    [651]
dtype: object